In [1]:
import pandas as pd
import os

--- Execução do Notebook Gold ---

In [2]:
# --- Configuração de Caminhos e Diretórios ---
path_silver = '../data/silver/dados_limpos.csv'
path_gold = '../data/gold/'
os.makedirs(path_gold, exist_ok=True)

In [3]:
# --- Carregar Dados da Camada Silver ---
try:
    df = pd.read_csv(path_silver)
    print(f"Dados limpos carregados: {df.shape}")
except FileNotFoundError:
    print(f"Erro: Arquivo '{path_silver}' não encontrado.")
    exit()

Dados limpos carregados: (55682, 31)


In [4]:
# ==========================================
# AGREGAÇÃO 1: Produção Anual por Cultura
# ==========================================
print("Calculando: Produção Anual por Cultura...")
prod_anual = df.groupby(['YEAR', 'Crop']).agg(
    Area_Total_Plantada=('Area', 'sum'),
    Producao_Total=('Production', 'sum')
).reset_index()

Calculando: Produção Anual por Cultura...


In [5]:
prod_anual['Rendimento_Ponderado'] = prod_anual.apply(
    lambda row: row['Producao_Total'] / row['Area_Total_Plantada'] if row['Area_Total_Plantada'] > 0 else 0,
    axis=1
)
prod_anual.to_csv(os.path.join(path_gold, 'producao_anual_cultura.csv'), index=False)

In [6]:
# ==========================================
# AGREGAÇÃO 2: Desempenho por Região e Cultura
# ==========================================
print("Calculando: Desempenho por Região e Cultura...")
desempenho_regiao = df.groupby(['State District', 'Crop']).agg(
    Area_Total=('Area', 'sum'),
    Producao_Total=('Production', 'sum'),
    Rendimento_Medio_Local=('Yield', 'mean')
).reset_index()
desempenho_regiao = desempenho_regiao.sort_values(by='Producao_Total', ascending=False)
desempenho_regiao.to_csv(os.path.join(path_gold, 'desempenho_regiao_cultura.csv'), index=False)

Calculando: Desempenho por Região e Cultura...


In [7]:
# ==========================================
# AGREGAÇÃO 3: Análise Sazonal e Climática
# ==========================================
print("Calculando: Análise Sazonal e Climática...")
# Usando os nomes de coluna corrigidos da Camada Bronze/Silver
analise_sazonal = df.groupby(['Season', 'Crop']).agg(
    Producao_Media=('Production', 'mean'),
    Rendimento_Medio=('Yield', 'mean'),
    Area_Media=('Area', 'mean'),
    # Agora usamos TEMP_JUN_SEP (temperatura) e Jun-Sep (precipitação)
    Temp_Media_Jun_Sep=('TEMP_JUN_SEP', 'mean'),
    Precipitacao_Media_Jun_Sep=('Jun-Sep', 'mean')
).reset_index()
analise_sazonal = analise_sazonal.sort_values(by='Rendimento_Medio', ascending=False)
analise_sazonal.to_csv(os.path.join(path_gold, 'analise_sazonal_clima.csv'), index=False)

Calculando: Análise Sazonal e Climática...


In [8]:
# ==========================================
# AGREGAÇÃO 4: Tendência de Rendimento Ano-a-Ano (Growth)
# ==========================================
print("Calculando: Tendência de Rendimento (Yield) Ano-a-Ano por Cultura...")
trend_rendimento = df.groupby(['Crop', 'YEAR']).agg(
    Rendimento_Medio=('Yield', 'mean')
).reset_index()
trend_rendimento = trend_rendimento.sort_values(by=['Crop', 'YEAR'])
trend_rendimento['Crescimento_Anual_Rendimento (%)'] = trend_rendimento.groupby('Crop')['Rendimento_Medio'].pct_change() * 100
trend_rendimento['Crescimento_Anual_Rendimento (%)'] = trend_rendimento['Crescimento_Anual_Rendimento (%)'].fillna(0)
trend_rendimento.to_csv(os.path.join(path_gold, 'tendencia_anual_rendimento.csv'), index=False)

Calculando: Tendência de Rendimento (Yield) Ano-a-Ano por Cultura...


In [9]:
# ==========================================
# AGREGAÇÃO 5: Benchmark de Desempenho Regional (vs. Média Nacional)
# ==========================================
print("Calculando: Benchmark de Desempenho Regional por Cultura...")
media_nacional_cultura = df.groupby('Crop')['Yield'].mean().to_frame(name='Rendimento_Medio_Nacional').reset_index()
media_regional_cultura = df.groupby(['State District', 'Crop']).agg(
    Rendimento_Medio_Local=('Yield', 'mean'),
    Producao_Total_Local=('Production', 'sum')
).reset_index()
benchmark_regional = pd.merge(
    media_regional_cultura,
    media_nacional_cultura,
    on='Crop'
)
benchmark_regional['Diferenca_vs_Media (%)'] = (
    (benchmark_regional['Rendimento_Medio_Local'] - benchmark_regional['Rendimento_Medio_Nacional']) /
     benchmark_regional['Rendimento_Medio_Nacional']
) * 100
benchmark_regional = benchmark_regional.sort_values(by='Diferenca_vs_Media (%)', ascending=False)
benchmark_regional.to_csv(os.path.join(path_gold, 'benchmark_regional_rendimento.csv'), index=False)

Calculando: Benchmark de Desempenho Regional por Cultura...


In [10]:
# ==========================================
# AGREGAÇÃO 6: Perfil Climático por Região e Cultura
# ==========================================
print("Calculando: Perfil Climático Médio por Região e Cultura...")
# Usando os nomes de coluna corrigidos da Camada Bronze/Silver
perfil_climatico = df.groupby(['State District', 'Crop']).agg(
    Rendimento_Medio=('Yield', 'mean'),
    Precipitacao_Anual_Media=('ANN', 'mean'),
    Precipitacao_Jun_Sep_Media=('Jun-Sep', 'mean'),
    Temp_Media_JanFeb=('TEMP_JAN_FEB', 'mean'),
    Temp_Media_MarMay=('TEMP_MAR_MAY', 'mean'),
    Temp_Media_JunSep=('TEMP_JUN_SEP', 'mean'),
    Temp_Media_OctDec=('TEMP_OCT_DEC', 'mean')
).reset_index()
perfil_climatico.to_csv(os.path.join(path_gold, 'perfil_climatico_regiao_cultura.csv'), index=False)

Calculando: Perfil Climático Médio por Região e Cultura...


In [11]:
# ==========================================
# AGREGAÇÃO 7: Análise de Volatilidade (Risco) do Rendimento
# ==========================================
print("Calculando: Análise de Volatilidade (Risco) do Rendimento...")
volatilidade_df = df.groupby(['State District', 'Crop']).agg(
    Rendimento_Medio=('Yield', 'mean'),
    Rendimento_Std_Dev=('Yield', 'std'),
    Contagem_Registros=('Yield', 'count')
).reset_index()

Calculando: Análise de Volatilidade (Risco) do Rendimento...


In [12]:
min_registros = 3
volatilidade_df = volatilidade_df[volatilidade_df['Contagem_Registros'] >= min_registros].copy()
print(f"Filtrando volatilidade para grupos com {min_registros} ou mais registros.")

volatilidade_df['Coef_Variacao_Rendimento (%)'] = (
    (volatilidade_df['Rendimento_Std_Dev'] / volatilidade_df['Rendimento_Medio']) * 100
)
volatilidade_df = volatilidade_df.fillna(0)
volatilidade_df = volatilidade_df.sort_values(by='Coef_Variacao_Rendimento (%)', ascending=False)
volatilidade_df.to_csv(os.path.join(path_gold, 'volatilidade_rendimento_regiao.csv'), index=False)

Filtrando volatilidade para grupos com 3 ou mais registros.
